In [9]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, PolynomialFeatures
from keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.keras.utils import load_img, img_to_array, array_to_img
from sklearn.metrics import mean_squared_error, mean_absolute_error
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
import random

from keras.models import Sequential, Model
from keras.regularizers import L1L2
from keras.layers import *
from keras.callbacks import TensorBoard, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping
from keras.optimizers import *
from keras import initializers
from keras import backend as K
from keras.layers import LSTM, InputLayer, Dense, Input, Flatten, concatenate, Reshape
from keras.callbacks import EarlyStopping
from random import randint
import pickle
import glob
import PIL

def rmse(y_true,y_pred):
    return np.sqrt(np.mean(np.square(y_pred-y_true)))

def mape(y_true, y_pred):
    mask = y_true != 0  # Create a mask for non-zero true values
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask]))

def smape(y_true, y_pred):
    numerator = np.abs(y_pred - y_true)
    denominator = (np.abs(y_true) + np.abs(y_pred)) / 2
    return np.mean(np.divide(numerator, denominator))

def getTT(region):
    return round((((region-100)//50) * -5.2222 ) + 147.11),round((((region%50)-2) * 5.2083)+20.458 )

def mean_absolute_percentage_error_revise(y_true, y_pred):
    # RMSE
    rmse = K.sqrt(K.mean(K.square(y_pred - y_true)))

    # SMAPE
    epsilon = 1e-13  # Small constant to avoid division by zero
    smape_numerator = tf.abs(y_true - y_pred)
    smape_denominator = tf.maximum(tf.abs(y_true) + tf.abs(y_pred), epsilon)
    smape = K.mean(smape_numerator / smape_denominator)

    alpha = 0.4 # hyperparameter, adjust alpha as needed
    combined_loss = alpha * rmse + (1 - alpha) * smape

    return combined_loss
def custom_metric(y_true, y_pred):
    return mean_absolute_percentage_error_revise(y_true, y_pred)

seed_value = 5566
tf.random.set_seed(seed_value)
np.random.seed(seed_value)
random.seed(seed_value)

In [10]:
region = 610
length = 16

epochs=150
min_lr=1e-8

myresults={}

train = pd.read_csv(f'csv/{region}.csv', index_col=0)
test = pd.read_csv(f'csv2/{region}.csv', index_col=0)

In [11]:
# polynomial + scale CSV

# create polynomial features
poly = PolynomialFeatures(2,include_bias=False)
train_poly = pd.DataFrame(poly.fit_transform(train),columns=poly.get_feature_names_out(train.columns))
test_poly = pd.DataFrame(poly.fit_transform(test),columns=poly.get_feature_names_out(test.columns))

# initialise scalers
minmaxscaler = MinMaxScaler(feature_range=(0.1,1))
minmaxscaler.fit(train_poly)
minmaxscalerY = MinMaxScaler(feature_range=(0.1,1))
minmaxscalerY.fit((train[[str(region)]]))

# scale train and test using scalers
scaled_train = pd.DataFrame(minmaxscaler.transform(train_poly),columns=train_poly.columns)
scaled_test = pd.DataFrame(minmaxscaler.transform(test_poly),columns=test_poly.columns)

In [12]:
# Create time series for CSV
## For testing
data = scaled_test.values
scaled_data = data

X2, y2 = [], []
sequence_length = 16  # X = 16 previous time steps

for i in range(len(scaled_data) - sequence_length):
    X2.append(scaled_data[i:i+sequence_length, :])  # Use all columns
    y2.append(scaled_data[i+sequence_length, 0])  # Assuming prediction is based on the first column

X2, y2 = np.array(X2), np.array(y2)

## For training
data = scaled_train.values
scaled_data = data

X, y = [], []
sequence_length = 16  # X = 16 previous time steps

for i in range(len(scaled_data) - sequence_length):
    X.append(scaled_data[i:i+sequence_length, :])  # Use all columns
    y.append(scaled_data[i+sequence_length, 0])  # Assuming prediction is based on the first column

X, y = np.array(X), np.array(y)

In [13]:
# Load images

def getalpha():
    theirimg = Image.open("alpha_image/alpha.png")
    alpha = theirimg.convert('RGBA').split()[-1]
    return alpha
alpha = getalpha()

images = []
for f in range(len(glob.glob("images/*.png"))):
    img = PIL.Image.open("images/%d.png"%(int(f)+1))
    r,g,b= img.split()
    bg = PIL.Image.new("RGBA",img.size,(255,255,255) + (255,))
    bg.paste(img,mask=alpha)
    bg = bg.convert('L')
    t,tt = getTT(region)
    bg = bg.crop((tt-32,t-32,tt+32,t+32))
    bg = bg.resize((9,9))
    x = img_to_array(bg)
    images.append(x)
    
images = np.array(images)
images = (images/255.0)


testImages = []
for f in range(len(glob.glob("images2/*.png"))):
    img = PIL.Image.open("images2/%d.png"%(int(f)+1))
    r,g,b= img.split()
    bg = PIL.Image.new("RGBA",img.size,(255,255,255) + (255,))
    bg.paste(img,mask=alpha)
    bg = bg.convert('L')
    t,tt = getTT(region)
    bg = bg.crop((tt-32,t-32,tt+32,t+32))
    bg = bg.resize((9,9))
    x = img_to_array(bg)
    testImages.append(x)

testImages = np.array(testImages)
testImages = (testImages/255.0)

print(testImages.shape,images.shape)

(2976, 9, 9, 1) (2976, 9, 9, 1)


In [14]:
# time series
# for testing
test_images_ts = []
window_size = 16
for i in range(0, testImages.shape[0] - window_size):
    window = testImages[i:i + window_size]
    test_images_ts.append(window)

# Convert the list to a NumPy array
test_images_ts = np.array(test_images_ts)
print(test_images_ts.shape)

# time series
# for training
train_images_ts = []
window_size = 16
for i in range(0, images.shape[0] - window_size):
    window = images[i:i + window_size]
    train_images_ts.append(window)

# Convert the list to a NumPy array
train_images_ts = np.array(train_images_ts)
print(train_images_ts.shape)

(2960, 16, 9, 9, 1)
(2960, 16, 9, 9, 1)


## CNN

In [15]:
# split data
train_size = int(len(X) * 0.90)  # 90% training, 10% val
X_train, X_val = X[:train_size], X[train_size:]
train_images_ts, val_images_ts = train_images_ts[:train_size], train_images_ts[train_size:] 
y_train, y_val = y[:train_size], y[train_size:]

In [16]:
#cnn
# Define the model
model = Sequential()
model.add(Conv3D(filters=32, kernel_size=(3, 3, 3), activation='relu', input_shape=(16, 9, 9, 1), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same'))
model.add(Conv3D(filters=64, kernel_size=(3, 3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same'))
model.add(Conv3D(filters=128, kernel_size=(3, 3, 3), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same'))
model.add(Flatten())

#output
model.add(Dense(units=1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss=mean_absolute_percentage_error_revise, metrics=[custom_metric])

2023-12-01 17:45:31.003171: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [17]:
reduce_lr = ReduceLROnPlateau(monitor='val_custom_metric', verbose=1,factor=0.5,patience=3, min_lr=min_lr)
checkpointer = ModelCheckpoint(filepath='weights/cnn_weights_dmv_%s_%d.hdf5'%('region',region), monitor='val_custom_metric',verbose=1,mode='min', save_best_only=True,save_weights_only=True)
earlyStoper = EarlyStopping(monitor='val_custom_metric', min_delta=0, patience=20, verbose=1, mode='min')
hist=model.fit(train_images_ts,y_train,batch_size=64,shuffle=False, epochs=epochs,validation_data=(val_images_ts,y_val),callbacks=[earlyStoper,checkpointer,reduce_lr],verbose=1)

Epoch 1/150
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing 

42/42 [==============================] - 19s 443ms/step - loss: 0.6584 - custom_metric: 0.6583 - val_loss: 0.6627 - val_custom_metric: 0.6663 - lr: 1.2500e-04
Epoch 15/150
42/42 [==============================] - ETA: 0s - loss: 0.6584 - custom_metric: 0.6583
Epoch 15: val_custom_metric did not improve from 0.13167

Epoch 15: ReduceLROnPlateau reducing learning rate to 6.25000029685907e-05.
42/42 [==============================] - 18s 440ms/step - loss: 0.6584 - custom_metric: 0.6583 - val_loss: 0.6627 - val_custom_metric: 0.6663 - lr: 1.2500e-04
Epoch 16/150
42/42 [==============================] - ETA: 0s - loss: 0.6584 - custom_metric: 0.6583
Epoch 16: val_custom_metric did not improve from 0.13167
42/42 [==============================] - 18s 432ms/step - loss: 0.6584 - custom_metric: 0.6583 - val_loss: 0.6627 - val_custom_metric: 0.6663 - lr: 6.2500e-05
Epoch 17/150
42/42 [==============================] - ETA: 0s - loss: 0.6584 - custom_metric: 0.6583
Epoch 17: val_custom_metric d

In [18]:
predictions = model.predict(test_images_ts)
myresults['cnn'] = {'current':{'rmse':rmse(y2,predictions), 'mape':mape(y2,predictions), 'smape':smape(y2,predictions)}}

model.load_weights(f'weights/cnn_weights_dmv_region_{region}.hdf5')
predictions = model.predict(test_images_ts)
myresults['cnn']['saved'] = {'rmse':rmse(y2,predictions), 'mape':mape(y2,predictions), 'smape':smape(y2,predictions)}

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
93/93 [==============================] - 2s 24ms/step


In [19]:
predictions = model.predict(test_images_ts)
print(rmse(y2,predictions))
print(mape(y2,predictions))
print(smape(y2,predictions))

93/93 [==============================] - 2s 23ms/step
0.06761103854302099
0.3009023688215202
0.29110174182689


In [20]:
model.load_weights(f'weights/cnn_weights_dmv_region_{region}.hdf5')
predictions = model.predict(test_images_ts)
print(rmse(y2,predictions))
print(mape(y2,predictions))
print(smape(y2,predictions))

93/93 [==============================] - 2s 23ms/step
0.06761103854302099
0.3009023688215202
0.29110174182689


## CNN + LSTM

In [21]:
# CNN branch
image_input = Input(shape=(16, 9, 9, 1), name='image_input')
cnn_branch = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(image_input)
cnn_branch = BatchNormalization()(cnn_branch)
cnn_branch = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(cnn_branch)  # Add max-pooling
cnn_branch = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(cnn_branch)
cnn_branch = BatchNormalization()(cnn_branch)
cnn_branch = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(cnn_branch)  # Add max-pooling
cnn_branch = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(cnn_branch)
cnn_branch = BatchNormalization()(cnn_branch)
cnn_branch = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(cnn_branch)  # Add max-pooling
cnn_branch = Flatten()(cnn_branch)
cnn_branch = Dense(64, activation='relu')(cnn_branch)
cnn_branch = Dense(16, activation='relu')(cnn_branch)

# LSTM branch
sequence_input = Input(shape=(X_train.shape[1], X_train.shape[2]), name='sequence_input') 
lstm_branch = LSTM(128, return_sequences=True)(sequence_input)

# Reshape the output to add a dummy dimension for Conv1D
lstm_branch_reshaped = Reshape((-1, 128))(lstm_branch)

# TCN layer
tcn_units = 128
dilation_factors = [1, 2, 4, 8, 16, 32, 64]
tcn_layer = lstm_branch_reshaped  # Initialize with the reshaped LSTM output
for dilation in dilation_factors:
    tcn_layer = Conv1D(filters=tcn_units, kernel_size=2, padding='causal', dilation_rate=dilation)(tcn_layer)
tcn_layer = GlobalAveragePooling1D()(tcn_layer)

# Concatenate the outputs of both branches
concatenated = Concatenate()([cnn_branch, tcn_layer])

# Fully connected layers for the concatenated output
final_output = Dense(128, activation='relu')(concatenated)
final_output = Dense(64, activation='relu')(final_output)
final_output = Dense(1, activation='sigmoid')(final_output)

# Create the model
model = Model(inputs=[image_input, sequence_input], outputs=final_output)

# Compile the model
model.compile(optimizer='adam', loss=mean_absolute_percentage_error_revise, metrics=[custom_metric])

In [22]:
reduce_lr = ReduceLROnPlateau(monitor='val_custom_metric', verbose=1,factor=0.5,patience=3, min_lr=min_lr)
checkpointer = ModelCheckpoint(filepath='weights/cnn_lstm_weights_dmv_%s_%d.hdf5'%('region',region), monitor='val_custom_metric',verbose=1,mode='min', save_best_only=True,save_weights_only=True)
earlyStoper = EarlyStopping(monitor='val_custom_metric', min_delta=0, patience=20, verbose=1, mode='min')
hist=model.fit([train_images_ts,X_train],y_train,batch_size=64,shuffle=False, epochs=epochs,validation_data=([val_images_ts,X_val],y_val),callbacks=[earlyStoper,checkpointer,reduce_lr],verbose=1)

Epoch 1/150
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
42/42 [==============================] - ETA: 0s - loss: 0.1376 - custom_metric: 0.1366WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7fb6c59bca70> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`

42/42 [==============================] - ETA: 0s - loss: 0.0844 - custom_metric: 0.0840
Epoch 18: val_custom_metric did not improve from 0.13042
42/42 [==============================] - 21s 509ms/step - loss: 0.0844 - custom_metric: 0.0840 - val_loss: 0.1748 - val_custom_metric: 0.1776 - lr: 0.0010
Epoch 19/150
42/42 [==============================] - ETA: 0s - loss: 0.0873 - custom_metric: 0.0868
Epoch 19: val_custom_metric improved from 0.13042 to 0.10538, saving model to weights/cnn_lstm_weights_dmv_region_610.hdf5
42/42 [==============================] - 22s 521ms/step - loss: 0.0873 - custom_metric: 0.0868 - val_loss: 0.0965 - val_custom_metric: 0.1054 - lr: 0.0010
Epoch 20/150
42/42 [==============================] - ETA: 0s - loss: 0.0828 - custom_metric: 0.0822
Epoch 20: val_custom_metric did not improve from 0.10538
42/42 [==============================] - 22s 529ms/step - loss: 0.0828 - custom_metric: 0.0822 - val_loss: 0.1388 - val_custom_metric: 0.1478 - lr: 0.0010
Epoch 21

Epoch 42/150
42/42 [==============================] - ETA: 0s - loss: 0.0737 - custom_metric: 0.0733
Epoch 42: val_custom_metric did not improve from 0.09965

Epoch 42: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
42/42 [==============================] - 24s 572ms/step - loss: 0.0737 - custom_metric: 0.0733 - val_loss: 0.0901 - val_custom_metric: 0.0997 - lr: 3.1250e-05
Epoch 43/150
42/42 [==============================] - ETA: 0s - loss: 0.0736 - custom_metric: 0.0731
Epoch 43: val_custom_metric improved from 0.09965 to 0.09964, saving model to weights/cnn_lstm_weights_dmv_region_610.hdf5
42/42 [==============================] - 24s 561ms/step - loss: 0.0736 - custom_metric: 0.0731 - val_loss: 0.0901 - val_custom_metric: 0.0996 - lr: 1.5625e-05
Epoch 44/150
42/42 [==============================] - ETA: 0s - loss: 0.0735 - custom_metric: 0.0731
Epoch 44: val_custom_metric improved from 0.09964 to 0.09963, saving model to weights/cnn_lstm_weights_dmv_region_610.hd


Epoch 63: ReduceLROnPlateau reducing learning rate to 2.4414063659605745e-07.
42/42 [==============================] - 20s 477ms/step - loss: 0.0733 - custom_metric: 0.0729 - val_loss: 0.0900 - val_custom_metric: 0.0996 - lr: 4.8828e-07
Epoch 64/150
42/42 [==============================] - ETA: 0s - loss: 0.0733 - custom_metric: 0.0729
Epoch 64: val_custom_metric improved from 0.09959 to 0.09959, saving model to weights/cnn_lstm_weights_dmv_region_610.hdf5
42/42 [==============================] - 20s 475ms/step - loss: 0.0733 - custom_metric: 0.0729 - val_loss: 0.0900 - val_custom_metric: 0.0996 - lr: 2.4414e-07
Epoch 65/150
42/42 [==============================] - ETA: 0s - loss: 0.0733 - custom_metric: 0.0729
Epoch 65: val_custom_metric improved from 0.09959 to 0.09959, saving model to weights/cnn_lstm_weights_dmv_region_610.hdf5
42/42 [==============================] - 20s 473ms/step - loss: 0.0733 - custom_metric: 0.0729 - val_loss: 0.0900 - val_custom_metric: 0.0996 - lr: 2.4414e

42/42 [==============================] - ETA: 0s - loss: 0.0733 - custom_metric: 0.0729
Epoch 86: val_custom_metric did not improve from 0.09959
42/42 [==============================] - 20s 475ms/step - loss: 0.0733 - custom_metric: 0.0729 - val_loss: 0.0900 - val_custom_metric: 0.0996 - lr: 1.0000e-08
Epoch 87/150
42/42 [==============================] - ETA: 0s - loss: 0.0733 - custom_metric: 0.0729
Epoch 87: val_custom_metric did not improve from 0.09959
42/42 [==============================] - 20s 465ms/step - loss: 0.0733 - custom_metric: 0.0729 - val_loss: 0.0900 - val_custom_metric: 0.0996 - lr: 1.0000e-08
Epoch 88/150
42/42 [==============================] - ETA: 0s - loss: 0.0733 - custom_metric: 0.0729
Epoch 88: val_custom_metric did not improve from 0.09959
42/42 [==============================] - 20s 473ms/step - loss: 0.0733 - custom_metric: 0.0729 - val_loss: 0.0900 - val_custom_metric: 0.0996 - lr: 1.0000e-08
Epoch 89/150
42/42 [==============================] - ETA: 0s 

Epoch 110/150
42/42 [==============================] - ETA: 0s - loss: 0.0733 - custom_metric: 0.0729
Epoch 110: val_custom_metric improved from 0.09959 to 0.09959, saving model to weights/cnn_lstm_weights_dmv_region_610.hdf5
42/42 [==============================] - 22s 533ms/step - loss: 0.0733 - custom_metric: 0.0729 - val_loss: 0.0900 - val_custom_metric: 0.0996 - lr: 1.0000e-08
Epoch 111/150
42/42 [==============================] - ETA: 0s - loss: 0.0733 - custom_metric: 0.0729
Epoch 111: val_custom_metric improved from 0.09959 to 0.09959, saving model to weights/cnn_lstm_weights_dmv_region_610.hdf5
42/42 [==============================] - 22s 514ms/step - loss: 0.0733 - custom_metric: 0.0729 - val_loss: 0.0900 - val_custom_metric: 0.0996 - lr: 1.0000e-08
Epoch 112/150
42/42 [==============================] - ETA: 0s - loss: 0.0733 - custom_metric: 0.0729
Epoch 112: val_custom_metric did not improve from 0.09959
42/42 [==============================] - 22s 530ms/step - loss: 0.0733

42/42 [==============================] - 20s 485ms/step - loss: 0.0733 - custom_metric: 0.0728 - val_loss: 0.0900 - val_custom_metric: 0.0996 - lr: 1.0000e-08
Epoch 134/150
42/42 [==============================] - ETA: 0s - loss: 0.0733 - custom_metric: 0.0728
Epoch 134: val_custom_metric did not improve from 0.09959
42/42 [==============================] - 20s 485ms/step - loss: 0.0733 - custom_metric: 0.0728 - val_loss: 0.0900 - val_custom_metric: 0.0996 - lr: 1.0000e-08
Epoch 135/150
42/42 [==============================] - ETA: 0s - loss: 0.0733 - custom_metric: 0.0728
Epoch 135: val_custom_metric did not improve from 0.09959
42/42 [==============================] - 23s 540ms/step - loss: 0.0733 - custom_metric: 0.0728 - val_loss: 0.0900 - val_custom_metric: 0.0996 - lr: 1.0000e-08
Epoch 136/150
42/42 [==============================] - ETA: 0s - loss: 0.0733 - custom_metric: 0.0728
Epoch 136: val_custom_metric improved from 0.09959 to 0.09959, saving model to weights/cnn_lstm_weigh

In [23]:
predictions = model.predict([test_images_ts,X2])
myresults['cnn_lstm'] = {'current':{'rmse':rmse(y2,predictions), 'mape':mape(y2,predictions), 'smape':smape(y2,predictions)}}

model.load_weights(f'weights/cnn_lstm_weights_dmv_region_{region}.hdf5')
predictions = model.predict([test_images_ts,X2])
myresults['cnn_lstm']['saved'] = {'rmse':rmse(y2,predictions), 'mape':mape(y2,predictions), 'smape':smape(y2,predictions)}

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
93/93 [==============================] - 3s 30ms/step


In [24]:
for m, v in myresults.items():
    print(m)
    print(v['current'])
    print(v['saved'])

cnn
{'rmse': 0.14062757014782157, 'mape': 0.9999999999906606, 'smape': 1.9999999999626485}
{'rmse': 0.06761103854302099, 'mape': 0.3009023688215202, 'smape': 0.29110174182689}
cnn_lstm
{'rmse': 0.0710776928859286, 'mape': 0.10488860486033376, 'smape': 0.1399533625197272}
{'rmse': 0.07107771635533701, 'mape': 0.10488928941919824, 'smape': 0.1399540914992473}


## CNN + LSTM + Micro

In [23]:
# split data
#train_size = int(len(X) * 0.90)  # 90% training, 10% val
#X_train, X_val = X[:train_size], X[train_size:]
#y_train, y_val = y[:train_size], y[train_size:]
#train_images_ts, val_images_ts = train_images_ts[:train_size], train_images_ts[train_size:] 

In [26]:
sequence_length = 16
my_m_input_train = pd.read_csv(f'micro/{region}.csv', index_col=0)
my_m_input_test = pd.read_csv(f'micro2/{region}.csv', index_col=0) # here
mydata_train = my_m_input_train.values
mydata_test = my_m_input_test.values

mscaler = MinMaxScaler(feature_range=(0.1,1))
mscaler.fit(mydata_train)
mydata_train = pd.DataFrame(mscaler.transform(mydata_train))
mydata_test = pd.DataFrame(mscaler.transform(mydata_test))

my_m_data_train = []
my_m_data_test = []

for i in range(len(mydata_train) - sequence_length):
    my_m_data_train.append(mydata_train[i:i+sequence_length])
    
for i in range(len(mydata_test) - sequence_length):
    my_m_data_test.append(mydata_test[i:i+sequence_length])

my_m_data_train = np.array(my_m_data_train)
my_m_data_test = np.array(my_m_data_test)

train_size = int(len(X) * 0.90)  # 90% training, 10% val
m_train_data, m_val_data = my_m_data_train[:train_size], my_m_data_train[train_size:]

In [27]:
# CNN branch
image_input = Input(shape=(16, 9, 9, 1), name='image_input')
cnn_branch = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(image_input)
cnn_branch = BatchNormalization()(cnn_branch)
cnn_branch = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(cnn_branch)  # Add max-pooling
cnn_branch = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(cnn_branch)
cnn_branch = BatchNormalization()(cnn_branch)
cnn_branch = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(cnn_branch)  # Add max-pooling
cnn_branch = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(cnn_branch)
cnn_branch = BatchNormalization()(cnn_branch)
cnn_branch = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(cnn_branch)  # Add max-pooling
cnn_branch = Flatten()(cnn_branch)
cnn_branch = Dense(64, activation='relu')(cnn_branch)
cnn_branch = Dense(16, activation='relu')(cnn_branch)

# LSTM branch
sequence_input = Input(shape=(X_train.shape[1], X_train.shape[2]), name='sequence_input') 
lstm_branch = LSTM(128, return_sequences=True)(sequence_input)


lstm_branch_reshaped = Reshape((-1, 128))(lstm_branch)

# TCN layer
tcn_units = 128
dilation_factors = [1, 2, 4, 8, 16, 32, 64]
tcn_layer = lstm_branch_reshaped  # Initialize with the reshaped LSTM output
for dilation in dilation_factors:
    tcn_layer = Conv1D(filters=tcn_units, kernel_size=2, padding='causal', dilation_rate=dilation)(tcn_layer)
tcn_layer = GlobalAveragePooling1D()(tcn_layer)

# M branch
micro_input = Input(shape=(m_train_data.shape[1], m_train_data.shape[2]), name='micro_input')

m_branch = LSTM(128, return_sequences=True)(micro_input)
m_branch = LSTM(128, return_sequences=True)(m_branch)
reshaped_m_branch = Flatten()(m_branch)

# Concatenate the outputs of both branches
concatenated = Concatenate()([cnn_branch, tcn_layer, reshaped_m_branch])

# Fully connected layers for the concatenated output
final_output = Dense(128, activation='relu')(concatenated)
final_output = Dense(64, activation='relu')(final_output)
final_output = Dense(1, activation='sigmoid')(final_output)

# Create the model
model = Model(inputs=[image_input, sequence_input, micro_input], outputs=final_output)

# Compile the model
model.compile(optimizer='adam', loss=mean_absolute_percentage_error_revise, metrics=[custom_metric])

In [28]:
reduce_lr = ReduceLROnPlateau(monitor='val_custom_metric', verbose=1,factor=0.5,patience=3, min_lr=min_lr)
checkpointer = ModelCheckpoint(filepath='weights/mm_weights_dmv_%s_%d.hdf5'%('region',region), monitor='val_custom_metric',verbose=1,mode='min', save_best_only=True,save_weights_only=True)
earlyStoper = EarlyStopping(monitor='val_custom_metric', min_delta=0, patience=20, verbose=1, mode='min')
hist=model.fit([train_images_ts,X_train, m_train_data],y_train,batch_size=64,shuffle=False, epochs=epochs,validation_data=([val_images_ts,X_val,m_val_data],y_val),callbacks=[earlyStoper,checkpointer,reduce_lr],verbose=1)

Epoch 1/150
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
42/42 [==============================] - ETA: 0s - loss: 0.2047 - custom_metric: 0.2036WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7fb6aa420c20> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`

Epoch 19/150
42/42 [==============================] - ETA: 0s - loss: 0.0753 - custom_metric: 0.0748
Epoch 19: val_custom_metric improved from 0.13048 to 0.12983, saving model to weights/mm_weights_dmv_region_610.hdf5
42/42 [==============================] - 27s 632ms/step - loss: 0.0753 - custom_metric: 0.0748 - val_loss: 0.1231 - val_custom_metric: 0.1298 - lr: 1.2500e-04
Epoch 20/150
42/42 [==============================] - ETA: 0s - loss: 0.0743 - custom_metric: 0.0738
Epoch 20: val_custom_metric improved from 0.12983 to 0.12150, saving model to weights/mm_weights_dmv_region_610.hdf5
42/42 [==============================] - 26s 608ms/step - loss: 0.0743 - custom_metric: 0.0738 - val_loss: 0.1139 - val_custom_metric: 0.1215 - lr: 1.2500e-04
Epoch 21/150
42/42 [==============================] - ETA: 0s - loss: 0.0752 - custom_metric: 0.0747
Epoch 21: val_custom_metric improved from 0.12150 to 0.12102, saving model to weights/mm_weights_dmv_region_610.hdf5
42/42 [=====================

42/42 [==============================] - 21s 498ms/step - loss: 0.0735 - custom_metric: 0.0730 - val_loss: 0.0913 - val_custom_metric: 0.1010 - lr: 3.1250e-05
Epoch 43/150
42/42 [==============================] - ETA: 0s - loss: 0.0735 - custom_metric: 0.0730
Epoch 43: val_custom_metric did not improve from 0.09996

Epoch 43: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
42/42 [==============================] - 22s 516ms/step - loss: 0.0735 - custom_metric: 0.0730 - val_loss: 0.0915 - val_custom_metric: 0.1011 - lr: 3.1250e-05
Epoch 44/150
42/42 [==============================] - ETA: 0s - loss: 0.0733 - custom_metric: 0.0729
Epoch 44: val_custom_metric did not improve from 0.09996
42/42 [==============================] - 22s 535ms/step - loss: 0.0733 - custom_metric: 0.0729 - val_loss: 0.0913 - val_custom_metric: 0.1009 - lr: 1.5625e-05
Epoch 45/150
42/42 [==============================] - ETA: 0s - loss: 0.0739 - custom_metric: 0.0734
Epoch 45: val_custom_metric

42/42 [==============================] - 25s 597ms/step - loss: 0.0731 - custom_metric: 0.0726 - val_loss: 0.0900 - val_custom_metric: 0.0998 - lr: 2.4414e-07
Epoch 65/150
42/42 [==============================] - ETA: 0s - loss: 0.0731 - custom_metric: 0.0726
Epoch 65: val_custom_metric did not improve from 0.09976
42/42 [==============================] - 26s 627ms/step - loss: 0.0731 - custom_metric: 0.0726 - val_loss: 0.0900 - val_custom_metric: 0.0998 - lr: 2.4414e-07
Epoch 66/150
42/42 [==============================] - ETA: 0s - loss: 0.0731 - custom_metric: 0.0726
Epoch 66: val_custom_metric did not improve from 0.09976

Epoch 66: ReduceLROnPlateau reducing learning rate to 1.2207031829802872e-07.
42/42 [==============================] - 27s 633ms/step - loss: 0.0731 - custom_metric: 0.0726 - val_loss: 0.0900 - val_custom_metric: 0.0998 - lr: 2.4414e-07
Epoch 67/150
42/42 [==============================] - ETA: 0s - loss: 0.0731 - custom_metric: 0.0726
Epoch 67: val_custom_metric

In [30]:
predictions = model.predict([test_images_ts,X2,my_m_data_test])
myresults['mm'] = {'current':{'rmse':rmse(y2,predictions), 'mape':mape(y2,predictions), 'smape':smape(y2,predictions)}}

model.load_weights(f'weights/mm_weights_dmv_region_{region}.hdf5')
predictions = model.predict([test_images_ts,X2,my_m_data_test])
myresults['mm']['saved'] = {'rmse':rmse(y2,predictions), 'mape':mape(y2,predictions), 'smape':smape(y2,predictions)}

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
93/93 [==============================] - 3s 31ms/step


In [31]:
for m, v in myresults.items():
    print(m)
    print(v['current'])
    print(v['saved'])

cnn
{'rmse': 0.14062757014782157, 'mape': 0.9999999999906606, 'smape': 1.9999999999626485}
{'rmse': 0.06761103854302099, 'mape': 0.3009023688215202, 'smape': 0.29110174182689}
cnn_lstm
{'rmse': 0.0710776928859286, 'mape': 0.10488860486033376, 'smape': 0.1399533625197272}
{'rmse': 0.07107771635533701, 'mape': 0.10488928941919824, 'smape': 0.1399540914992473}
mm
{'rmse': 0.07112235226415754, 'mape': 0.1017006358619653, 'smape': 0.13693325228076456}
{'rmse': 0.07116729543423228, 'mape': 0.1016288126363456, 'smape': 0.13696658682089638}


## CNN + Micro

In [33]:
# CNN branch
image_input = Input(shape=(16, 9, 9, 1), name='image_input')
cnn_branch = Conv3D(32, (3, 3, 3), activation='relu', padding='same')(image_input)
cnn_branch = BatchNormalization()(cnn_branch)
cnn_branch = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(cnn_branch)  # Add max-pooling
cnn_branch = Conv3D(64, (3, 3, 3), activation='relu', padding='same')(cnn_branch)
cnn_branch = BatchNormalization()(cnn_branch)
cnn_branch = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(cnn_branch)  # Add max-pooling
cnn_branch = Conv3D(128, (3, 3, 3), activation='relu', padding='same')(cnn_branch)
cnn_branch = BatchNormalization()(cnn_branch)
cnn_branch = MaxPooling3D(pool_size=(2, 2, 2), strides=(2, 2, 2), padding='same')(cnn_branch)  # Add max-pooling
cnn_branch = Flatten()(cnn_branch)
cnn_branch = Dense(64, activation='relu')(cnn_branch)
cnn_branch = Dense(16, activation='relu')(cnn_branch)

# M branch
micro_input = Input(shape=(m_train_data.shape[1], m_train_data.shape[2]), name='micro_input')

m_branch = LSTM(128, return_sequences=True)(micro_input)
m_branch = LSTM(128, return_sequences=True)(m_branch)
reshaped_m_branch = Flatten()(m_branch)

# Concatenate the outputs of both branches
concatenated = Concatenate()([cnn_branch, reshaped_m_branch])

# Fully connected layers for the concatenated output
final_output = Dense(128, activation='relu')(concatenated)
final_output = Dense(64, activation='relu')(final_output)
final_output = Dense(1, activation='sigmoid')(final_output)

# Create the model
model = Model(inputs=[image_input, micro_input], outputs=final_output)

# Compile the model
model.compile(optimizer='adam', loss=mean_absolute_percentage_error_revise, metrics=[custom_metric])

In [36]:
reduce_lr = ReduceLROnPlateau(monitor='val_custom_metric', verbose=1,factor=0.5,patience=3, min_lr=min_lr)
checkpointer = ModelCheckpoint(filepath='weights/cnn_mm_weights_dmv_%s_%d.hdf5'%('region',region), monitor='val_custom_metric',verbose=1,mode='min', save_best_only=True,save_weights_only=True)
earlyStoper = EarlyStopping(monitor='val_custom_metric', min_delta=0, patience=20, verbose=1, mode='min')
hist=model.fit([train_images_ts, m_train_data],y_train,batch_size=64,shuffle=False, epochs=epochs,validation_data=([val_images_ts,m_val_data],y_val),callbacks=[earlyStoper,checkpointer,reduce_lr],verbose=1)

Epoch 1/150
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
42/42 [==============================] - ETA: 0s - loss: 0.1432 - custom_metric: 0.1424WARNING:tensorflow:AutoGraph could not transform <function Model.make_test_function.<locals>.test_function at 0x7fb698362cb0> and will run it as-is.
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`

42/42 [==============================] - ETA: 0s - loss: 0.0823 - custom_metric: 0.0818
Epoch 18: val_custom_metric did not improve from 0.13931
42/42 [==============================] - 23s 558ms/step - loss: 0.0823 - custom_metric: 0.0818 - val_loss: 0.1349 - val_custom_metric: 0.1404 - lr: 0.0010
Epoch 19/150
42/42 [==============================] - ETA: 0s - loss: 0.0783 - custom_metric: 0.0779
Epoch 19: val_custom_metric improved from 0.13931 to 0.12882, saving model to weights/cnn_mm_weights_dmv_region_610.hdf5
42/42 [==============================] - 23s 537ms/step - loss: 0.0783 - custom_metric: 0.0779 - val_loss: 0.1221 - val_custom_metric: 0.1288 - lr: 0.0010
Epoch 20/150
42/42 [==============================] - ETA: 0s - loss: 0.0851 - custom_metric: 0.0846
Epoch 20: val_custom_metric did not improve from 0.12882
42/42 [==============================] - 21s 508ms/step - loss: 0.0851 - custom_metric: 0.0846 - val_loss: 0.1264 - val_custom_metric: 0.1327 - lr: 0.0010
Epoch 21/1

Epoch 42/150
42/42 [==============================] - ETA: 0s - loss: 0.0734 - custom_metric: 0.0729
Epoch 42: val_custom_metric improved from 0.10021 to 0.10018, saving model to weights/cnn_mm_weights_dmv_region_610.hdf5
42/42 [==============================] - 20s 468ms/step - loss: 0.0734 - custom_metric: 0.0729 - val_loss: 0.0905 - val_custom_metric: 0.1002 - lr: 3.1250e-05
Epoch 43/150
42/42 [==============================] - ETA: 0s - loss: 0.0734 - custom_metric: 0.0729
Epoch 43: val_custom_metric did not improve from 0.10018

Epoch 43: ReduceLROnPlateau reducing learning rate to 1.5625000742147677e-05.
42/42 [==============================] - 22s 527ms/step - loss: 0.0734 - custom_metric: 0.0729 - val_loss: 0.0905 - val_custom_metric: 0.1002 - lr: 3.1250e-05
Epoch 44/150
42/42 [==============================] - ETA: 0s - loss: 0.0734 - custom_metric: 0.0729
Epoch 44: val_custom_metric improved from 0.10018 to 0.10013, saving model to weights/cnn_mm_weights_dmv_region_610.hdf5
4

42/42 [==============================] - ETA: 0s - loss: 0.0732 - custom_metric: 0.0728
Epoch 64: val_custom_metric improved from 0.10012 to 0.10012, saving model to weights/cnn_mm_weights_dmv_region_610.hdf5
42/42 [==============================] - 21s 497ms/step - loss: 0.0732 - custom_metric: 0.0728 - val_loss: 0.0905 - val_custom_metric: 0.1001 - lr: 2.4414e-07
Epoch 65/150
42/42 [==============================] - ETA: 0s - loss: 0.0732 - custom_metric: 0.0728
Epoch 65: val_custom_metric improved from 0.10012 to 0.10012, saving model to weights/cnn_mm_weights_dmv_region_610.hdf5

Epoch 65: ReduceLROnPlateau reducing learning rate to 1.2207031829802872e-07.
42/42 [==============================] - 24s 574ms/step - loss: 0.0732 - custom_metric: 0.0728 - val_loss: 0.0905 - val_custom_metric: 0.1001 - lr: 2.4414e-07
Epoch 66/150
42/42 [==============================] - ETA: 0s - loss: 0.0732 - custom_metric: 0.0728
Epoch 66: val_custom_metric improved from 0.10012 to 0.10012, saving mo

42/42 [==============================] - 19s 456ms/step - loss: 0.0732 - custom_metric: 0.0728 - val_loss: 0.0905 - val_custom_metric: 0.1001 - lr: 1.0000e-08
Epoch 85/150
42/42 [==============================] - ETA: 0s - loss: 0.0732 - custom_metric: 0.0728
Epoch 85: val_custom_metric improved from 0.10012 to 0.10012, saving model to weights/cnn_mm_weights_dmv_region_610.hdf5
42/42 [==============================] - 19s 455ms/step - loss: 0.0732 - custom_metric: 0.0728 - val_loss: 0.0905 - val_custom_metric: 0.1001 - lr: 1.0000e-08
Epoch 86/150
42/42 [==============================] - ETA: 0s - loss: 0.0732 - custom_metric: 0.0728
Epoch 86: val_custom_metric improved from 0.10012 to 0.10012, saving model to weights/cnn_mm_weights_dmv_region_610.hdf5
42/42 [==============================] - 19s 454ms/step - loss: 0.0732 - custom_metric: 0.0728 - val_loss: 0.0905 - val_custom_metric: 0.1001 - lr: 1.0000e-08
Epoch 87/150
42/42 [==============================] - ETA: 0s - loss: 0.0732 - 

42/42 [==============================] - ETA: 0s - loss: 0.0732 - custom_metric: 0.0728
Epoch 106: val_custom_metric improved from 0.10012 to 0.10012, saving model to weights/cnn_mm_weights_dmv_region_610.hdf5
42/42 [==============================] - 19s 452ms/step - loss: 0.0732 - custom_metric: 0.0728 - val_loss: 0.0905 - val_custom_metric: 0.1001 - lr: 1.0000e-08
Epoch 107/150
42/42 [==============================] - ETA: 0s - loss: 0.0732 - custom_metric: 0.0728
Epoch 107: val_custom_metric improved from 0.10012 to 0.10012, saving model to weights/cnn_mm_weights_dmv_region_610.hdf5
42/42 [==============================] - 18s 440ms/step - loss: 0.0732 - custom_metric: 0.0728 - val_loss: 0.0905 - val_custom_metric: 0.1001 - lr: 1.0000e-08
Epoch 108/150
42/42 [==============================] - ETA: 0s - loss: 0.0732 - custom_metric: 0.0728
Epoch 108: val_custom_metric improved from 0.10012 to 0.10012, saving model to weights/cnn_mm_weights_dmv_region_610.hdf5
42/42 [=================

42/42 [==============================] - 20s 471ms/step - loss: 0.0732 - custom_metric: 0.0728 - val_loss: 0.0905 - val_custom_metric: 0.1001 - lr: 1.0000e-08
Epoch 128/150
42/42 [==============================] - ETA: 0s - loss: 0.0732 - custom_metric: 0.0728
Epoch 128: val_custom_metric improved from 0.10012 to 0.10012, saving model to weights/cnn_mm_weights_dmv_region_610.hdf5
42/42 [==============================] - 20s 488ms/step - loss: 0.0732 - custom_metric: 0.0728 - val_loss: 0.0905 - val_custom_metric: 0.1001 - lr: 1.0000e-08
Epoch 129/150
42/42 [==============================] - ETA: 0s - loss: 0.0732 - custom_metric: 0.0728
Epoch 129: val_custom_metric improved from 0.10012 to 0.10012, saving model to weights/cnn_mm_weights_dmv_region_610.hdf5
42/42 [==============================] - 19s 461ms/step - loss: 0.0732 - custom_metric: 0.0728 - val_loss: 0.0905 - val_custom_metric: 0.1001 - lr: 1.0000e-08
Epoch 130/150
42/42 [==============================] - ETA: 0s - loss: 0.07

Epoch 149/150
42/42 [==============================] - ETA: 0s - loss: 0.0732 - custom_metric: 0.0728
Epoch 149: val_custom_metric improved from 0.10012 to 0.10012, saving model to weights/cnn_mm_weights_dmv_region_610.hdf5
42/42 [==============================] - 19s 441ms/step - loss: 0.0732 - custom_metric: 0.0728 - val_loss: 0.0905 - val_custom_metric: 0.1001 - lr: 1.0000e-08
Epoch 150/150
42/42 [==============================] - ETA: 0s - loss: 0.0732 - custom_metric: 0.0728
Epoch 150: val_custom_metric improved from 0.10012 to 0.10012, saving model to weights/cnn_mm_weights_dmv_region_610.hdf5
42/42 [==============================] - 19s 441ms/step - loss: 0.0732 - custom_metric: 0.0728 - val_loss: 0.0905 - val_custom_metric: 0.1001 - lr: 1.0000e-08


In [37]:
predictions = model.predict([test_images_ts,my_m_data_test])
myresults['cnn_mm'] = {'current':{'rmse':rmse(y2,predictions), 'mape':mape(y2,predictions), 'smape':smape(y2,predictions)}}

model.load_weights(f'weights/cnn_mm_weights_dmv_region_{region}.hdf5')
predictions = model.predict([test_images_ts,my_m_data_test])
myresults['cnn_mm']['saved'] = {'rmse':rmse(y2,predictions), 'mape':mape(y2,predictions), 'smape':smape(y2,predictions)}

Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: 'arguments' object has no attribute 'posonlyargs'
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
93/93 [==============================] - 2s 26ms/step


In [38]:
for m, v in myresults.items():
    print(m)
    print(v['current'])
    print(v['saved'])

cnn
{'rmse': 0.14062757014782157, 'mape': 0.9999999999906606, 'smape': 1.9999999999626485}
{'rmse': 0.06761103854302099, 'mape': 0.3009023688215202, 'smape': 0.29110174182689}
cnn_lstm
{'rmse': 0.0710776928859286, 'mape': 0.10488860486033376, 'smape': 0.1399533625197272}
{'rmse': 0.07107771635533701, 'mape': 0.10488928941919824, 'smape': 0.1399540914992473}
mm
{'rmse': 0.07112235226415754, 'mape': 0.1017006358619653, 'smape': 0.13693325228076456}
{'rmse': 0.07116729543423228, 'mape': 0.1016288126363456, 'smape': 0.13696658682089638}
cnn_mm
{'rmse': 0.07100980197722886, 'mape': 0.1049854271977903, 'smape': 0.13989076810742931}
{'rmse': 0.07100980197722886, 'mape': 0.1049854271977903, 'smape': 0.13989076810742931}
